#So basically this is the extension part of the project i have done while learning llm engineeering
the flow is simple

#pre-existing:

*   to scrap the data from the company site and extracting usefull links

*  passing the scrapped data to the model (gpt-4o-mini) with proper system and user promt to get clean bronchure in markdown

#I added up
(  just thoght what if i somehow write the bronchure in the pdf with some styling and formatting)


*   i used the library weasyprint which allowed me to write the content on the pdf with using html and css
*  i passed the bronchure content to the model(gpt-4o-mini/gpt-4o) to gt the styled html and css to embed in the weasyprint

#📓note:
try creating mutiple times to get choose god one out of it








In [93]:
# # Core packages
# !pip install requests
# !pip install python-dotenv
# !pip install beautifulsoup4


# !pip install openai
# !pip install weasyprint


In [94]:
# imports
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [95]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [96]:
company_url=input("Enter theh company url:")
company_name=input("Enter the company name:")

Enter theh company url:https://huggingface.co/
Enter the company name:Huggingface


In [97]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [98]:
site = Website(company_url)
site.links


['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/Qwen/Qwen-Image-Edit',
 '/deepseek-ai/DeepSeek-V3.1-Base',
 '/google/gemma-3-270m',
 '/deepseek-ai/DeepSeek-V3.1',
 '/tencent/Hunyuan-GameCraft-1.0',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/Qwen/Qwen-Image-Edit',
 '/spaces/zerogpu-aoti/wan2-2-fp8da-aoti-faster',
 '/spaces/AIDC-AI/Ovis2.5-9B',
 '/spaces/Qwen/Qwen-Image',
 '/spaces',
 '/datasets/nvidia/Granary',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/nvidia/Llama-Nemotron-VLM-Dataset-v1',
 '/datasets/allenai/WildChat-4.8M',
 '/datasets/FreedomIntelligence/medical-o1-reasoning-SFT',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/tr

In [99]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [100]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}



In [101]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [102]:
print(get_links_user_prompt(site))

Here is the list of links on the website of https://huggingface.co/ - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
/
/models
/datasets
/spaces
/docs
/enterprise
/pricing
/login
/join
/spaces
/models
/Qwen/Qwen-Image-Edit
/deepseek-ai/DeepSeek-V3.1-Base
/google/gemma-3-270m
/deepseek-ai/DeepSeek-V3.1
/tencent/Hunyuan-GameCraft-1.0
/models
/spaces/enzostvs/deepsite
/spaces/Qwen/Qwen-Image-Edit
/spaces/zerogpu-aoti/wan2-2-fp8da-aoti-faster
/spaces/AIDC-AI/Ovis2.5-9B
/spaces/Qwen/Qwen-Image
/spaces
/datasets/nvidia/Granary
/datasets/fka/awesome-chatgpt-prompts
/datasets/nvidia/Llama-Nemotron-VLM-Dataset-v1
/datasets/allenai/WildChat-4.8M
/datasets/FreedomIntelligence/medical-o1-reasoning-SFT
/datasets
/join
/pricing#endpoints
/pricing#spaces
/pricing
/enterprise
/enterprise
/enterprise
/enterprise
/enterprise
/

In [103]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [104]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [105]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [106]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [107]:
get_brochure_user_prompt(company_name, company_url)

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/#huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}]}


'You are looking at a company called: Huggingface\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nQwen/Qwen-Image-Edit\nUpdated\n4 days ago\n•\n23.5k\n•\n1.02k\ndeepseek-ai/DeepSeek-V3.1-Base\nUpdated\nabout 2 hours ago\n•\n10.9k\n•\n838\ngoogle/gemma-3-270m\nUpdated\n8 days ago\n•\n57.7k\n•\n578\ndeepseek-ai/DeepSeek-V3.1\nUpdated\nabout 2 hours ago\n•\n7.8k\n•\n399\ntencent/Hunyuan-GameCraft-1.0\nUpdated\n3 days ago\n•\n153\n•\n449\nBrowse 1M+ models\nSpaces\nRunning\n12.2k\n12.2k\nD

In [108]:

def create_brochure(company_name, url):
    global result
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [109]:

create_brochure(company_name, company_url)


Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'join page', 'url': 'https://huggingface.co/join'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}]}


# Hugging Face Brochure

## Company Overview
**Hugging Face** is at the forefront of the artificial intelligence (AI) revolution, creating an inclusive and collaborative community that builds the future of machine learning (ML). We empower users to explore, create, and share AI models, datasets, and applications on a comprehensive platform that enhances collaboration and accelerates innovation.

### Our Mission
To democratize AI and nurture an open-source culture that fosters creativity and collaboration among all skill levels in the ML community.

## Core Offerings
- **Models:** Explore over 1 million AI models, including the latest advancements and trending projects like Qwen and DeepSeek.
- **Datasets:** Access a vast collection of 250,000 datasets essential for training and deploying machine learning algorithms.
- **Spaces:** Create and run AI applications effortlessly, fostering real-time collaboration.
- **Enterprise Solutions:** Tailored solutions offering robust security, priority support, and advanced features for organizations aiming to scale.

## Company Culture
At Hugging Face, we pride ourselves on our community-centric ethos. We embrace an open-source philosophy, encouraging innovation and collaboration. Our work environment is designed to support creativity where every voice is valued. We foster an inclusive culture that invites diverse perspectives, making it a great place for all individuals passionate about AI and machine learning.

### Values:
- **Inclusivity:** Encouraging participation from all backgrounds and skill levels.
- **Collaboration:** Building a community where knowledge is shared and collective progress is prioritized.
- **Innovation:** Continuously pushing the boundaries of what is possible with AI.

## Customer Base
With over 50,000 organizations relying on Hugging Face, including industry leaders like Google, Microsoft, and Amazon, our platform is trusted by enterprises around the globe. Our tools have been adopted across various sectors, enabling teams to deploy state-of-the-art AI without compromising on security or efficiency.

## Career Opportunities
**Join the Hugging Face Family!**
We are always looking for passionate individuals who want to shape the future of AI. Explore our latest openings and be a part of a team that values innovation and collaboration.

### Current Openings:
- Software Engineers
- Data Scientists
- Community Managers
- Product Managers

### Benefits of Working with Us:
- Work in a flexible and dynamic environment.
- Engage in meaningful projects that make a real impact.
- Participate in continuous learning and development opportunities.

---

For more information about our services, community involvement, or career opportunities, visit our website at [Hugging Face](https://huggingface.co). Let's build the future of AI together!

In [110]:
print(result)

# Hugging Face Brochure

## Company Overview
**Hugging Face** is at the forefront of the artificial intelligence (AI) revolution, creating an inclusive and collaborative community that builds the future of machine learning (ML). We empower users to explore, create, and share AI models, datasets, and applications on a comprehensive platform that enhances collaboration and accelerates innovation.

### Our Mission
To democratize AI and nurture an open-source culture that fosters creativity and collaboration among all skill levels in the ML community.

## Core Offerings
- **Models:** Explore over 1 million AI models, including the latest advancements and trending projects like Qwen and DeepSeek.
- **Datasets:** Access a vast collection of 250,000 datasets essential for training and deploying machine learning algorithms.
- **Spaces:** Create and run AI applications effortlessly, fostering real-time collaboration.
- **Enterprise Solutions:** Tailored solutions offering robust security, prior

In [111]:
system_prompt_design='''
You are a creative AI brochure architect. Your mission is to convert any company's formatted website or structured input into a visually impressive, contemporary, and print-ready two-page brochure solely in HTML + embedded CSS, ready for seamless conversion with WeasyPrint.

Instructions:

Use modern, design-forward layout principles: experiment with grids, cards, visually-contrasted modules or sidebars, vertical or horizontal info areas, hero headings, large blocks, etc.

Color palette: Primarily light blues, blue-whites, sky blues, with darker or brighter accents as needed for contrast, consistency, and visual engagement.

Include CSS effects as appropriate for print (e.g., subtle drop-shadows, borders, large bold headings, soft gradients, colored side stripes, or title blocks).

Organize all company input into beautiful structured sections—break out features, values, sections, side notes, or testimonials as suited to the content.

Strictly limit to two pages: Arrange content and adjust sizing so everything fits elegantly within exactly two print pages. Place
<div class="page-break"></div> between the pages.

Typography: Use only embedded fonts—modern, legible, e.g. Segoe UI, Arial, sans-serif, with bold, size, tracking, and case effects for headings.

No external images or asset links: Create all visuals with CSS, use colored blocks or shaped divs for illustrations/logos if you wish.

No explanations or markdown—only the final HTML and CSS data.

Ensure the HTML output is instantly usable for PDF rendering with WeasyPrint, A4/Letter size.

Input:
Formatted company/website content.

Output:
Complete, attractive, fully styled 2-page HTML+CSS brochure, with the above requirements.Do NOT wrap your output in triple backticks or any markdown fencing. Output raw HTML only.

Make use of all layout, design, and creative freedom, while ensuring clarity and alignment with modern print design.'''
user_prompt_design="this is the markdowned broonchure skeleton "
user_prompt_design+=result
print(user_prompt_design)


this is the markdowned broonchure skeleton # Hugging Face Brochure

## Company Overview
**Hugging Face** is at the forefront of the artificial intelligence (AI) revolution, creating an inclusive and collaborative community that builds the future of machine learning (ML). We empower users to explore, create, and share AI models, datasets, and applications on a comprehensive platform that enhances collaboration and accelerates innovation.

### Our Mission
To democratize AI and nurture an open-source culture that fosters creativity and collaboration among all skill levels in the ML community.

## Core Offerings
- **Models:** Explore over 1 million AI models, including the latest advancements and trending projects like Qwen and DeepSeek.
- **Datasets:** Access a vast collection of 250,000 datasets essential for training and deploying machine learning algorithms.
- **Spaces:** Create and run AI applications effortlessly, fostering real-time collaboration.
- **Enterprise Solutions:** Tailore

In [112]:
m1="gpt-4o-mini"
m2="gpt-4o"
#standard m=m1(recommended)
m=m1
choice=input("which model you want to use :\n 1)gpt-4o-mini : cheaper and standard result \n2) gpt-4o  : costlies creative and professional brnchure \n")
if choice=='2':
  m=m2
else:
  m=m1
respoonse=openai.chat.completions.create(
    model=m,
    messages=[
        {"role": "system", "content": system_prompt_design},
        {"role":"user","content":user_prompt_design}]
)
print(respoonse.choices[0].message.content)

which model you want to use :
 1)gpt-4o-mini : cheaper and standard result 
2) gpt-4o  : costlies creative and professional brnchure 
1
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Hugging Face Brochure</title>
    <style>
        body {
            font-family: 'Segoe UI', Arial, sans-serif;
            margin: 0;
            padding: 0;
            background: #f4f9fc;
            color: #333;
            line-height: 1.5;
        }
        .container {
            max-width: 800px;
            margin: 0 auto;
            padding: 20px;
        }
        h1, h2, h3 {
            color: #0078d4;
            margin: 10px 0;
        }
        h1 {
            font-size: 2.5em;
            text-align: center;
            border-bottom: 2px solid #0078d4;
            padding-bottom: 10px;
        }
        h2 {
            font-size: 1.8em;
            margin-top: 20px;
            

In [113]:

from weasyprint import HTML

html_content = respoonse.choices[0].message.content
HTML(string=html_content).write_pdf((company_name)+".pdf")


DEBUG:fontTools.ttLib.ttFont:Reading 'maxp' table from disk
DEBUG:fontTools.ttLib.ttFont:Decompiling 'maxp' table
DEBUG:fontTools.subset.timer:Took 0.005s to load 'maxp'
DEBUG:fontTools.subset.timer:Took 0.000s to prune 'maxp'
INFO:fontTools.subset:maxp pruned
DEBUG:fontTools.ttLib.ttFont:Reading 'cmap' table from disk
DEBUG:fontTools.ttLib.ttFont:Decompiling 'cmap' table
DEBUG:fontTools.ttLib.ttFont:Reading 'post' table from disk
DEBUG:fontTools.ttLib.ttFont:Decompiling 'post' table
DEBUG:fontTools.subset.timer:Took 0.008s to load 'cmap'
DEBUG:fontTools.subset.timer:Took 0.000s to prune 'cmap'
INFO:fontTools.subset:cmap pruned
INFO:fontTools.subset:fpgm dropped
INFO:fontTools.subset:prep dropped
INFO:fontTools.subset:cvt  dropped
INFO:fontTools.subset:kern dropped
DEBUG:fontTools.subset.timer:Took 0.000s to load 'post'
DEBUG:fontTools.subset.timer:Took 0.000s to prune 'post'
INFO:fontTools.subset:post pruned
INFO:fontTools.subset:GPOS dropped
INFO:fontTools.subset:GSUB dropped
DEBUG:f